# Batch run models

In [1]:
%load_ext lab_black
import os, multiprocessing, time, papermill, sys
from IPython.display import clear_output
from tqdm import tqdm
from meta import ModelConfig, make_batch_cfg

# nest_asyncio.apply()
# Gen OG random seed: [int(random.random() * 1e5) for x in range(10)]

## Make configs

In [2]:
batch_name = "error_injection_timing_test"
batch_output_dir = f"models/batch_run/{batch_name}/"

param_grid = {
    "rng_seed": [75429, 32572, 81590, 15999, 75629, 79044, 70079, 41958, 50626, 92328],
    "inject_error_ticks": [2, 11],
}

static_hpar = {
    "tf_root": "/home/jupyter/tf",
    "ort_units": 119,
    "pho_units": 250,
    "sem_units": 2446,
    "hidden_os_units": 500,
    "hidden_op_units": 100,
    "hidden_ps_units": 500,
    "hidden_sp_units": 500,
    "pho_cleanup_units": 20,
    "sem_cleanup_units": 50,
    "pho_noise_level": 0.0,
    "sem_noise_level": 0.0,
    "activation": "sigmoid",
    "tau": 1 / 3,
    "max_unit_time": 4.0,
    "output_ticks": 11,
    "learning_rate": 0.01,
    "zero_error_radius": 0.1,
    "n_mil_sample": 2.0,
    "batch_size": 100,
    "save_freq": 10,
    "batch_name": batch_name,
}

batch_cfgs = make_batch_cfg(
    batch_name, batch_output_dir, static_hpar, param_grid, "hs04_phase2.ipynb"
)

init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
init from scratch
Batch config saved to models/batch_run/error_injection_timing_test/
There are 20 models in this batch


## Run batch
- Can run on Jupyter Lab
- Cannot run on VSCode

In [3]:
# Run
def run_batch(cfg):
    """
    Using papermill to run parameterized notebook
    To prevent overwriting, set default overwrite to False if needed
    """

    print("Running model {}".format(cfg["sn"]))
    os.makedirs(cfg["model_folder"], exist_ok=True)
    papermill.execute_notebook(
        cfg["in_notebook"],
        cfg["out_notebook"],
        parameters=cfg["params"],
    )

    clear_output()


# Run in parallel pool

# with multiprocessing.Pool(2) as pool:
#     pool.map(run_batch, batch_cfgs)


# Run in series
for cfg in tqdm(batch_cfgs):
    run_batch(cfg)


print("Done")

100%|██████████| 20/20 [4:30:03<00:00, 810.18s/it]

Done


Compile and save results

In [ ]:
# Cannot execute together with parallel run for some reason... maybe due to nest_asynio... forgot how to fix...
# cfgs, df = parse_batch_results(batch_cfgs)
# df.to_csv(batch_output_dir + "bcdf.csv")
# cfgs.to_csv(batch_output_dir + "cfgs.csv")

In [4]:
import pandas as pd
import sqlite3

con = sqlite3.connect(batch_output_dir + "batch_results.sqlite")
cur = con.cursor()
df = pd.read_sql_query("SELECT * from strain", con)

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('strain',), ('grain',), ('taraban',)]


Shutdown compute engine

In [ ]:
time.sleep(30)
print('shuting down')
!sudo poweroff